In [1]:
import tensorflow as tf
import cv2
import numpy as np

In [2]:
image_shape = (64 , 64)

In [3]:
model = tf.keras.models.Sequential([
          tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(image_shape[0], image_shape[1], 3)),
                                    tf.keras.layers.Conv2D(32, (5,5), activation='relu'),
                                    tf.keras.layers.MaxPool2D(2, 2),
                                    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
                                    tf.keras.layers.MaxPool2D(2, 2),
                                    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
                                    tf.keras.layers.MaxPool2D(2, 2),
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(256, activation='relu'),
                                    tf.keras.layers.Dropout(0.2),
                                    tf.keras.layers.Dense(29, activation = 'softmax')
])

In [4]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])

In [5]:
model.load_weights('final_best_weights.h5')

In [6]:
# Used to map prediction number to it's corresponding letter
from string import ascii_lowercase
dic = {i[0]: i[1] for i in enumerate(ascii_lowercase)}
# Extra signs to help convert from live feed to text in the future
dic[26] = 'delete'
dic[27] = 'nothing'
dic[28] = 'space'
dic[29] = 'not sure'

In [7]:
def predict_letter(test, certainity_threshold = 0.5):
    img = cv2.resize(test, image_shape)
    newimg = np.asarray(img)
    pixels = newimg.astype('float32')

    pixels = pixels.reshape(1, image_shape[0], image_shape[1] ,3 )
    predict = model.predict(pixels)

    letter_index = np.argmax(predict)
    
    if predict[0][letter_index] < certainity_threshold:  #if not certain
        return 29 , None
    else:
        return letter_index, predict[0][letter_index]

In [11]:
# set Width and Height of output Screen
frameWidth = 480
frameHeight = 480
   
# capturing Video from Webcam
cap = cv2.VideoCapture(0)
cap.set(3, frameWidth)
cap.set(4, frameHeight)

True

In [ ]:
ENTER_KEY_CODE = 13
is_first_image = True
box = None # Subset of the image that the model will check for sign in

while True:
    success, img = cap.read()
    
    if not success:
        cv2.destroyAllWindows()
        print("Couldn't Start cam, make sure to start the video capture")
        break
        
    if is_first_image:
        box = cv2.selectROI('ROI', img, fromCenter = False)
        is_first_image = False
        cv2.destroyWindow('ROI')

    # Vertices of bounding box
    pt1 = box[:2]
    pt2 = tuple((box[0] + box[2] ,box[1] + box[3]))
    

    prediction , certainity = predict_letter(img[pt1[1] : pt2[1], pt1[0]: pt2[0]] ,
                                             certainity_threshold=0.6)
    
    if certainity == None:
        text = 'Not Sure'
    else:
        certainity =  format(certainity * 100, '.2f')
        text = dic[prediction] + ' , ' + str(certainity) + '%'
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(img, text , (10,450), font, 1, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.putText(img, 'Press "S" to change bounding box',(frameWidth - 130, 435) , font, 0.5, (200, 0, 0), 1, cv2.LINE_AA)
    cv2.putText(img, 'Press "Enter" to exit',(frameWidth - 130, 455) , font, 0.5, (200, 0, 0), 1, cv2.LINE_AA)
    
    
    #draw boundary box
    bounded = cv2.rectangle(img, pt1, pt2 ,(255, 0, 0) , 2)
    cv2.imshow("Result", bounded)
    
    key = cv2.waitKeyEx(1)
    if key != -1:
        if key == ord('s'): #Select ROI again
            cv2.destroyWindow("Result")
            is_first_image = True
        elif key == ENTER_KEY_CODE:
            cv2.destroyWindow("Result")
            break
        
        
    
cap.release()
cv2.destroyAllWindows()